# Projeto 1 - Ciência dos Dados

Nome: Vítor Nucci Sevá Pereira

Nome: Leonardo Almodi dos Reis

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\theok\OneDrive\Área de Trabalho\Projeto 1 Cdados 2023-2\Proj1_LEOTHEO_CDADOS


Carregando a base de dados com os tweets classificados manualmente:

In [3]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

,Mensagem,Target
0,Os livros e suas capas estão ótimos. Bonitos e...,2
1,Como pode uma editora aproveitar o sucesso do ...,0
2,"Muito ruim, uma série de repetições desnecessá...",0
3,Cansativo. O kindle não facilita as idas e vin...,2
4,O final decepcionante da trilogia me fez arrep...,0


In [4]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Target
0,Livros e caixa vieram danificados. Parte exter...,2
1,"A ""obra"" é uma fraude completa. O autor é um a...",0
2,Trata de modo superficial o tema da velocidade...,0
3,"Descreve um relacionamento abusivo, que é bem ...",0
4,"O box chegou com amassados, riscos e com uma e...",2


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

Inicialmente, iremos chamar as targets de seguintes nomes no código:
0 -> Sobre livros = Neutro;
1 -> Sobre autor = Positivo;
2 -> Sobre sites e encomendas = Negativo;

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### • Limpando as reviews:

In [5]:
import re 

def cleanup(text):

    # Função de limpeza muito simples que troca alguns sinais básicos por espaços
    
    punctuation = '[´"!-.:?;$''/\|@,><%&*()_+^~`}{+=#*n]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed



In [6]:

for i in range(len (train.Mensagem)-1):
    train.Mensagem[i] = cleanup(train.Mensagem[i])


C:\Users\theok\AppData\Local\Temp\ipykernel_55596\388858104.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.Mensagem[i] = cleanup(train.Mensagem[i])


### • Criando listas com todas as palavras de cada classificação:

In [7]:
lista_total = []
lista_neutro = []
lista_positivos = []
lista_negativos = []

for i in range(len (train.Mensagem)-1):
    # Separando os rows do DataFrame
    x = train.Mensagem[i].lower().split()
    

    t = 0
    while t < len(x):
        lista_total.append(x[t])
        t+=1

    if train.iloc[i,1] == 0:
        n = 0
        while n < len(x):
            lista_neutro.append(x[n])
            n+=1

    elif train.iloc[i,1] == 1:
        j = 0
        while j < len(x):
            lista_positivos.append(x[j])
            j+=1
        
    elif train.iloc[i,1] == 2:
        k = 0
        while k < len(x):
            lista_negativos.append(x[k])
            k+=1



### • Criando um pd.series para cada categoria:

In [8]:
serie_total = pd.Series(lista_total)

serie_positivos = pd.Series(lista_positivos)

serie_negativos = pd.Series(lista_negativos)

serie_neutro = pd.Series(lista_neutro)


### • Frequências absolutas:

In [9]:
total_absoluta = serie_total.value_counts()

positivos_absoluta = serie_positivos.value_counts()

negativos_absoluta = serie_negativos.value_counts()

neutro_absoluta = serie_neutro.value_counts()

### • Frequências relativas:

In [10]:
total_relativa = serie_total.value_counts(True)

positivos_relativa = serie_positivos.value_counts(True)

negativos_relativa = serie_negativos.value_counts(True)

neutro_relativa = serie_neutro.value_counts(True)

### • Probabilidades:

In [11]:
P_positivo = len(positivos_absoluta)/len(total_absoluta)
P_negativo = len(negativos_absoluta)/len(total_absoluta)
P_neutro = len(neutro_absoluta)/len(total_absoluta)
lista_classificacao = []

for mensagem in train.Mensagem:
    
    lista_palavras = mensagem.split()

    P_positivo = 1
    P_negativo = 1
    P_neutro = 1

    for palavra in lista_palavras:
            if palavra in neutro_absoluta:
                P_neutro *= (neutro_absoluta[palavra]+1)/(len(neutro_absoluta)+ len(total_absoluta))
            else:
                P_neutro *= 1/(len(neutro_absoluta) + len(total_absoluta))

            if palavra in positivos_absoluta:
                P_positivo *= (positivos_absoluta[palavra]+1)/(len(positivos_absoluta) + len(total_absoluta))
            else:
                P_positivo  *= 1/(len(positivos_absoluta) + len(total_absoluta))

            if palavra in negativos_absoluta:
                P_negativo *= (negativos_absoluta[palavra]+1)/(len(negativos_absoluta) + len(total_absoluta))
            else:
                P_negativo *= 1/(len(negativos_absoluta) + len(total_absoluta))


    if P_positivo > P_negativo and P_positivo > P_neutro:
        lista_classificacao.append(1)
    elif P_negativo > P_positivo and P_negativo > P_neutro:
        lista_classificacao.append(2)
    else:
        lista_classificacao.append(0)

train['Target (código)'] = lista_classificacao
train

,Mensagem,Target,Target (código)
0,Os livros e suas capas estão ótimos Boitos e s...,2,2
1,Como pode uma editora aproveitar o sucesso do ...,0,0
2,Muito ruim uma série de repetições desecessári...,0,0
3,Casativo O kidle ão facilita as idas e vidas o...,2,0
4,O fial decepcioate da trilogia me fez arrepede...,0,0
...,...,...,...
295,O livro é excelete porém chegou todo amassado ...,2,2
296,O livro é de autoajuda e deveria ser vedido co...,0,0
297,A história em si é boa mas é bem casativa Tete...,0,0
298,Estava tetado há dias cotiuar ledo sem que fic...,0,0


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [12]:
# Limpando os dados de teste
for i in range(len (test.Mensagem)-1):
    test.Mensagem[i] = cleanup(test.Mensagem[i])



# Criando as listas de cada categoria
lista_total_teste = []
lista_neutro_teste = []
lista_positivos_teste = []
lista_negativos_teste = []

for i in range(len (test.Mensagem)-1):
    x = test.Mensagem[i].lower().split()
    

    t = 0
    while t < len(x):
        lista_total_teste.append(x[t])
        t+=1

    if test.iloc[i,1] == 0:
        n = 0
        while n < len(x):
            lista_neutro_teste.append(x[n])
            n+=1

    elif test.iloc[i,1] == 1:
        j = 0
        while j < len(x):
            lista_positivos_teste.append(x[j])
            j+=1
        
    elif test.iloc[i,1] == 2:
        k = 0
        while k < len(x):
            lista_negativos_teste.append(x[k])
            k+=1



# Criando as séries de cada categoria
serie_total_teste = pd.Series(lista_total_teste)

serie_positivos_teste = pd.Series(lista_positivos_teste)

serie_negativos_teste = pd.Series(lista_negativos_teste)

serie_neutro_teste = pd.Series(lista_neutro_teste)



# Criando as tabelas de frequência absoluta e relativa de cada categoria
total_absoluta_teste = serie_total_teste.value_counts()
positivos_absoluta_teste = serie_positivos_teste.value_counts()
negativos_absoluta_teste = serie_negativos_teste.value_counts()
neutro_absoluta_teste = serie_neutro_teste.value_counts()

total_relativa_teste = serie_total_teste.value_counts(True)
positivos_relativa_teste = serie_positivos_teste.value_counts(True)
negativos_relativa_teste = serie_negativos_teste.value_counts(True)
neutro_relativa_teste = serie_neutro_teste.value_counts(True)


# Probabilidades

P_positivo_teste = len(positivos_absoluta_teste)/len(total_absoluta_teste)
P_negativo_teste = len(negativos_absoluta_teste)/len(total_absoluta_teste)
P_neutro_teste = len(neutro_absoluta_teste)/len(total_absoluta_teste)
lista_classificacao_teste = []



for mensagem in test.Mensagem:
    
    lista_palavras_teste = mensagem.split()

    P_positivo_teste = 1
    P_negativo_teste = 1
    P_neutro_teste = 1

    for palavra in lista_palavras_teste:
            if palavra in neutro_absoluta_teste:
                P_neutro_teste *= (neutro_absoluta_teste[palavra]+1)/(len(neutro_absoluta_teste)+ len(total_absoluta_teste))
            else:
                P_neutro_teste *= 1/(len(neutro_absoluta_teste) + len(total_absoluta_teste))

            if palavra in positivos_absoluta_teste:
                P_positivo_teste *= (positivos_absoluta_teste[palavra]+1)/(len(positivos_absoluta_teste) + len(total_absoluta_teste))
            else:
                P_positivo_teste  *= 1/(len(positivos_absoluta_teste) + len(total_absoluta_teste))

            if palavra in negativos_absoluta_teste:
                P_negativo_teste *= (negativos_absoluta_teste[palavra]+1)/(len(negativos_absoluta_teste) + len(total_absoluta_teste))
            else:
                P_negativo_teste *= 1/(len(negativos_absoluta_teste) + len(total_absoluta_teste))


    if P_positivo_teste > P_negativo_teste and P_positivo_teste > P_neutro_teste:
        lista_classificacao_teste.append(1)
    elif P_negativo_teste > P_positivo_teste and P_negativo_teste > P_neutro_teste:
        lista_classificacao_teste.append(2)
    else:
        lista_classificacao_teste.append(0)

test['Target (código)'] = lista_classificacao_teste



C:\Users\theok\AppData\Local\Temp\ipykernel_55596\4024697688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Mensagem[i] = cleanup(test.Mensagem[i])


In [13]:
test.head(5)

,Mensagem,Target,Target (código)
0,Livros e caixa vieram daificados Parte extera ...,2,2
1,A obra é uma fraude completa O autor é um astr...,0,0
2,Trata de modo superficial o tema da velocidade...,0,0
3,Descreve um relacioameto abusivo que é bem dif...,0,0
4,O box chegou com amassados riscos e com uma es...,2,2


In [14]:
lista_classificacao_correta = test.Target

tabela_relativa = pd.crosstab(lista_classificacao_correta, lista_classificacao_teste, rownames=['Target'], colnames=['Target (código)'], normalize=True)
display(tabela_relativa)
print('\n')


# Porcentagem de verdadeiros positivos:
verdadeiro_positivo = tabela_relativa.iloc[1,1]

# Porcentagem de verdadeiros negativos:
verdadeiro_negativo = tabela_relativa.iloc[2,2]

# Porcentagem de verdadeiros neutros:
verdadeiro_neutro = tabela_relativa.iloc[0,0]

# Porcentagem de falsos neutros:
falso_neutro = tabela_relativa.iloc[1,0] + tabela_relativa.iloc[2,0]

# Porcentagem de falsos positivos:
falso_positivo = tabela_relativa.iloc[0,1] + tabela_relativa.iloc[2,1]

# Porcentagem de falsos negativos:
falso_negativo = tabela_relativa.iloc[0,2] + tabela_relativa.iloc[1,2]

# Tabela de porcentagem de acertos e erros:
tabela_erros_acertos = pd.DataFrame({'Verdadeiro Positivo': [f"{verdadeiro_positivo*100:.2f}%"], 'Verdadeiro Negativo': [f"{verdadeiro_negativo*100:.2f}%"], 'Verdadeiro Neutro': [f"{verdadeiro_neutro*100:.2f}%"], 'Falso Neutro': [f"{falso_neutro*100:.2f}%"], 'Falso Positivo': [f"{falso_positivo*100:.2f}%"], 'Falso Negativo': [f"{falso_negativo*100:.2f}%"]})
display(tabela_erros_acertos)
print('\n')


# Porcentagem de acertos quando código da neutro:
neutros_acertos = tabela_relativa.iloc[0,0]/(tabela_relativa.iloc[:,0].sum())
print(f'A porcantegem de acerto quando o código fala que é neutro, é de: {neutros_acertos.round(4)*100}%')

# Porcentagem de acertos quando código da positivo:
positivos_acertos = tabela_relativa.iloc[1,1]/(tabela_relativa.iloc[:,1].sum())
print(f'A porcantegem de acerto quando o código fala que é positivo, é de: {positivos_acertos.round(4)*100}%')

# Porcentagem de acertos quando código da negativo:
negativos_acertos = tabela_relativa.iloc[2,2]/(tabela_relativa.iloc[:,2].sum())
print(f'A porcantegem de acerto quando o código fala que é negativo, é de: {negativos_acertos.round(4)*100}%')



# Acurácia:
acuracia = tabela_relativa.iloc[0,0] + tabela_relativa.iloc[1,1] + tabela_relativa.iloc[2,2]
print('\n')
print(f'A acurácia é de: {acuracia.round(4)*100}%')


Target (código),0,1,2
Target,,,
0,0.575,0.000,0.000
1,0.140,0.025,0.005
2,0.030,0.000,0.225


,Verdadeiro Positivo,Verdadeiro Negativo,Verdadeiro Neutro,Falso Neutro,Falso Positivo,Falso Negativo
0,2.50%,22.50%,57.50%,17.00%,0.00%,0.50%




A porcantegem de acerto quando o código fala que é neutro, é de: 77.18%
A porcantegem de acerto quando o código fala que é positivo, é de: 100.0%
A porcantegem de acerto quando o código fala que é negativo, é de: 97.83%


A acurácia é de: 82.5%


___
### Concluindo

A partir da análise dos resultados obtidos, conseguimos observar que a porcentagem de acerto quando o código fala que é do livro é de 100%, isso se deve ao fato de que na nossa base de dados, obtivemos muitas reviews sobre o livro em si. Contudo, tivemos uma porcentagem relativamente baixa comparada com as demais em relação às criticas voltadas ao autor, isso pode ser explicado a partir dos reviews dessa target, tendo em vista que diversas vezes não falavam-se apenas do autor em específico, mas também de sua obra.
Contudo, a acurácia exata do nosso projeto foi de 82.5%, relativamente alta, indicando que em geral o resultado obtido no projeto foi relevante e confiável.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

In [15]:
from sklearn.model_selection import train_test_split

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Bigramas (enunciado do projeto)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**